In [1]:
import clickhouse_connect
import pandas as pd
import polars as pl

pl.Config.set_fmt_str_lengths(200)
pl.Config.set_fmt_float("full")

polars.config.Config

### Direct connect to Cryptohouse
Connect to cryptohouse with client to do some initial querying and result aggregation.


### Limitations
- Can only do 1000 blocks, the direct access limits are still being figured out.


In [2]:
# Create a connection to the ClickHouse server
client = clickhouse_connect.get_client(
    host='crypto-clickhouse.clickhouse.com',
    username='crypto',
    password='',
    secure=True
)

In [3]:
# Execute the query and return as a pandas dataframe
query: str = """
SELECT
  *
FROM
  solana.transactions ARRAY
  JOIN accounts AS account
WHERE
  arrayExists(
    x -> x.1 IN [ 
        'Fc8bpeCMifWYv97pQ3k5xDvd98nuVg6yAaZrwmy4RRp6', -- zeta dex address
        'pythWSnswVUd12oZpeFP8e9CVaEqJg25g1Vtc2biRsT' -- pyth price feed
    ],
    accounts
  )
 // The block_slot range 282,300,000 to 282,900,000 is about 3 days and is a known range where probems were being experienced
#  AND block_slot > 282899000
#  AND block_slot < 282900000
  AND block_timestamp >= now() - INTERVAL 5 MINUTE
  AND account.2 = true -- Filter for signer accounts only
"""

In [4]:
results: pd.DataFrame = client.query_df(query)

In [5]:
df = pl.from_pandas(results)

In [6]:
df.group_by('status', 'log_messages').agg(
    pl.len().alias('count')).sort(by='count', descending=True)

status,log_messages,count
str,list[str],u32
"""0""","[""Program ComputeBudget111111111111111111111111111111 invoke [1]"", ""Program ComputeBudget111111111111111111111111111111 success"", … ""Program ZETAxsqBRek56DhiGXrn75yj2NHU3aYUnxvHXpkf3aD failed: custom program error: 0x1819""]",402
"""1""","[""Program 11111111111111111111111111111111 invoke [1]"", ""Program 11111111111111111111111111111111 success"", … ""Program 11111111111111111111111111111111 success""]",56
"""0""","[""Program HDwcJBJXjL9FpJ7UBsYBtaDjsBUhuLCUYoz3zr8SWWaQ invoke [1]"", ""Program log: Instruction: WriteEncodedVaa"", … ""Program HDwcJBJXjL9FpJ7UBsYBtaDjsBUhuLCUYoz3zr8SWWaQ failed: custom program error: 0xbbb""]",13
"""1""","[""Program HDwcJBJXjL9FpJ7UBsYBtaDjsBUhuLCUYoz3zr8SWWaQ invoke [1]"", ""Program log: Instruction: WriteEncodedVaa"", … ""Program ComputeBudget111111111111111111111111111111 success""]",8
"""1""","[""Program pythWSnswVUd12oZpeFP8e9CVaEqJg25g1Vtc2biRsT invoke [1]"", ""Program log: Instruction: UpdatePriceFeed"", … ""Program ComputeBudget111111111111111111111111111111 success""]",8
…,…,…
"""1""","[""Program ComputeBudget111111111111111111111111111111 invoke [1]"", ""Program ComputeBudget111111111111111111111111111111 success"", … ""Program ZETAxsqBRek56DhiGXrn75yj2NHU3aYUnxvHXpkf3aD success""]",1
"""1""","[""Program ComputeBudget111111111111111111111111111111 invoke [1]"", ""Program ComputeBudget111111111111111111111111111111 success"", … ""Program ZETAxsqBRek56DhiGXrn75yj2NHU3aYUnxvHXpkf3aD success""]",1
"""1""","[""Program ComputeBudget111111111111111111111111111111 invoke [1]"", ""Program ComputeBudget111111111111111111111111111111 success"", … ""Program ZETAxsqBRek56DhiGXrn75yj2NHU3aYUnxvHXpkf3aD success""]",1


In [7]:
# blocks with highest error rates
df.group_by('block_slot', 'status').agg(pl.len().alias('count')).sort(
    by='count', descending=True).filter(pl.col('status') == "0").head(10)

block_slot,status,count
i64,str,u32
284218699,"""0""",34
284218767,"""0""",27
284218698,"""0""",24
284218551,"""0""",23
284218481,"""0""",20
284218477,"""0""",19
284218242,"""0""",19
284218427,"""0""",18
284218342,"""0""",18


In [8]:
df.group_by('block_slot', 'status').agg(pl.len().alias('count')).sort(
    by='count', descending=True).filter(pl.col('status') == "1").head(10)

block_slot,status,count
i64,str,u32
284218418,"""1""",28
284218381,"""1""",24
284218583,"""1""",22
284218353,"""1""",16
284218339,"""1""",16
284218762,"""1""",16
284218768,"""1""",16
284218239,"""1""",16
284218727,"""1""",16


### What do failed messages look like?

In [9]:
high_fail_block = df.filter(pl.col('block_slot') == 284194723).group_by(
    'block_slot', 'status').agg(pl.len().alias('count')).sort(by='count', descending=True)
high_fail_block

block_slot,status,count
i64,str,u32


In [10]:
fail_df = df.filter(pl.col('block_slot') == 284194723)

In [11]:
fail_df.filter(pl.col('status') == "0").group_by('log_messages').agg(
    pl.len().alias('count')).sort(by='count', descending=True)

log_messages,count
list[str],u32


In [12]:
for i in range(len(fail_df.filter(pl.col('status') == "0").group_by('log_messages').agg(pl.len().alias('count')).sort(by='count', descending=True).to_dicts())):
    print(fail_df.filter(pl.col('status') == "0").group_by('log_messages').agg(
        pl.len().alias('count')).sort(by='count', descending=True).to_dicts()[i])

In [13]:
# Filter the DataFrame where any log_message contains 'vaa'.
filtered_df = fail_df.with_columns(
    default_match=pl.col("log_messages").list.eval(
        pl.element().str.contains("Error", literal=True, strict=True))
)

In [14]:
explode_filter = filtered_df.select('index', 'status', 'log_messages', 'default_match').explode(
    'log_messages', 'default_match').filter(pl.col('default_match') == True)

In [15]:
# we just get a generic invalid oracle update value.
explode_filter.group_by('log_messages', 'default_match').agg(
    pl.len().alias('count'))

log_messages,default_match,count
str,bool,u32


### jito blocks

In [16]:
# List of jito tip accounts
jito_tip_accts = [
    '96gYZGLnJYVFmbjzopPSU6QiEV5fGqZNyN9nmNhvrZU5',
    'HFqU5x63VTqvQss8hp11i4wVV8bD44PvwucfZ2bU7gRe',
    'Cw8CFyM9FkoMi7K7Crf6HNQqf4uEMzpKw6QNghXLvLkY',
    'ADaUMid9yfUytqMBgopwjb2DTLSokTSzL1zt6iGPaS49',
    'DfXygSm4jCyNCybVYYK6DwvWqjKee8pbDmJGcLWNDXjh',
    'ADuUkR4vqLUMWXxW9gh6D6L8pMSawimctcNZ5pGwDcEt',
    'DttWaMuVvTiduZRnguLF7jNxTgiMBZ1hyAumKUiL2KRL',
    '3AVi9Tg9Uo68tJfuvoKvqKNWKkC5wPdSSdeBnizKZ6jT'
]

In [19]:
jito_df = (
    df.explode('accounts')
    .with_columns(
    pl.col("accounts").struct.field("pubkey").alias("jit_acct")
    )
).filter(pl.col('jit_acct').is_in(jito_tip_accts)).select('block_slot').unique()

In [26]:
filtered_block_df = df.with_columns(
    pl.col('block_slot').is_in(jito_df.to_pandas()['block_slot'].to_list()).alias('jito_block')
)


In [43]:
(
    filtered_block_df
    .group_by('block_slot', 'status').agg(
    pl.len().alias('count'),
    pl.col('jito_block').first().alias('jito_block')
    )
    .filter(pl.col('jito_block') == False)
    # .group_by('block_slot', 'jito_bl  ock').agg(
    #     pl.col('count')
    # )
    .sort(by='count', descending=True)
    ).group_by('status', 'jito_block').agg(
        pl.len().alias('block_count'))


status,jito_block,block_count
str,bool,u32
"""1""",false,146
"""0""",false,46


In [44]:
(
    filtered_block_df
    .group_by('block_slot', 'status').agg(
    pl.len().alias('count'),
    pl.col('jito_block').first().alias('jito_block')
    )
    .filter(pl.col('jito_block') == True)
    # .group_by('block_slot', 'jito_block').agg(
    #     pl.col('count')
    # )
    .sort(by='count', descending=True)
    ).group_by('status', 'jito_block').agg(
        pl.len().alias('block_count'))

status,jito_block,block_count
str,bool,u32
"""0""",true,2
"""1""",true,28
